# 5 W's detection from sample text with spacy

In [109]:
import spacy

# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [110]:
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import re

In [111]:


#text = "Yet Harry Potter was still there, asleep at the moment, but not for long. His Aunt Petunia was awake and it was her shrill voice that made the first noise of the day."
text="EVERY morning, when I open my eyes to the so-called “new day,” I feel like closing them again, staying in bed,and not getting up. But I can’t do that. Later I go to the garden and see birds flying around. The same place where I met my husband Jacob 3 years ago."
 
doc = nlp(text)

for token in doc:
    print(token.text,'->',token.dep_,'->',token.pos_)

EVERY -> det -> DET
morning -> npadvmod -> NOUN
, -> punct -> PUNCT
when -> advmod -> ADV
I -> nsubj -> PRON
open -> advcl -> VERB
my -> poss -> DET
eyes -> dobj -> NOUN
to -> prep -> ADP
the -> det -> DET
so -> advmod -> ADV
- -> punct -> PUNCT
called -> amod -> VERB
“ -> punct -> PUNCT
new -> amod -> ADJ
day -> pobj -> NOUN
, -> punct -> PUNCT
” -> punct -> PUNCT
I -> nsubj -> PRON
feel -> ROOT -> VERB
like -> prep -> SCONJ
closing -> pcomp -> VERB
them -> dobj -> PRON
again -> advmod -> ADV
, -> punct -> PUNCT
staying -> advcl -> VERB
in -> prep -> ADP
bed -> pobj -> NOUN
, -> punct -> PUNCT
and -> cc -> CCONJ
not -> neg -> PART
getting -> conj -> VERB
up -> prt -> ADP
. -> punct -> PUNCT
But -> cc -> CCONJ
I -> nsubj -> PRON
ca -> aux -> VERB
n’t -> neg -> PART
do -> ROOT -> AUX
that -> dobj -> DET
. -> punct -> PUNCT
Later -> advmod -> ADV
I -> nsubj -> PRON
go -> ROOT -> VERB
to -> prep -> ADP
the -> det -> DET
garden -> pobj -> NOUN
and -> cc -> CCONJ
see -> conj -> VERB
birds -

In [112]:
for token in doc:
    
    if token.pos_=='NOUN':
      print(token.text)

morning
eyes
day
bed
garden
birds
place
husband
years


In [113]:
from spacy import displacy 
displacy.render(doc, style='dep',jupyter=True)

In [114]:
print('Who - subject')
for token in doc:
    
    if (token.dep_=='nsubj' and token.pos_=='PRON'):
      print(token.text)
    

Who - subject
I
I
I
I
I


In [115]:
print('what - object')
for token in doc:
   
    if (token.dep_=='dobj'):
      print(token.text)

what - object
eyes
them
that
husband


In [116]:
print('when')
for token in doc:
    
    if (token.dep_=='npadvmod'):
      print(token.text, "-->",token.dep_, "-->",token.pos_)

when
morning --> npadvmod --> NOUN
years --> npadvmod --> NOUN


In [117]:
matcher = Matcher(nlp.vocab) 

pattern = [{'DEP':'nummod','OP':"?"}, 
           {'POS':'NUM'},
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'} 
        ] 
                               
matcher.add("matching", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

3 years


In [118]:
print('where')
for token in doc:
    
    if (token.dep_=='pobj'):
      print(token.text)

where
day
bed
garden


In [119]:
print('whom')
for token in doc:
    
    if (token.dep_=='appos'):
      print(token.text, "-->",token.dep_, "-->",token.pos_)

whom
Jacob --> appos --> PROPN


In [120]:
matcher = Matcher(nlp.vocab) 

pattern = [
           {'DEP':'poss' }, 
           {'DEP':'dobj'} ,
           {'DEP':'appos'}
        ] 
                               
matcher.add("matching", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

my husband Jacob


zalando flair

In [121]:
#pip install flair

In [122]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [123]:
tagger = SequenceTagger.load('ner')

2021-07-06 09:28:07,486 --------------------------------------------------------------------------------
2021-07-06 09:28:07,489 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-07-06 09:28:07,490  - The most current version of the model is automatically downloaded from there.
2021-07-06 09:28:07,494  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-07-06 09:28:07,495 --------------------------------------------------------------------------------
2021-07-06 09:28:07,846 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [124]:
sentence = Sentence(text)

In [125]:
tagger.predict(sentence)

for entity in sentence.get_spans('ner'):
    print(entity)

Span [57]: "Jacob"   [− Labels: PER (0.9955)]


# Count no. of events

In [126]:
nlp = spacy.load("en_core_web_sm")
text1="After the fall of the Western Roman Empire in the 5th century, the Franks, like other post-Roman Western Europeans, emerged as a tribal confederacy in the Middle Rhine-Weser region, among the territory soon to be called Austrasia (the 'eastern land'), the northeastern portion of the future Kingdom of the Merovingian Franks."
doc = nlp(text1)
for ent in doc.ents:
  print("{} -> {}".format(ent.text,ent.label_))

the Western Roman Empire -> LOC
the 5th century -> DATE
post-Roman Western Europeans -> EVENT
the Middle Rhine-Weser -> LOC
Austrasia -> GPE
Kingdom -> GPE
the Merovingian Franks -> ORG


In [127]:
doc = nlp(text1)
for ent in filter(lambda e: e.label_=='DATE',doc.ents):
  print(ent.text)

the 5th century


In [128]:
text2='The Holy Roman Empire controlled the land for nearly 1,000 years, and between the mid-14th century and 1918 most of Slovenia was under Habsburg rule. In 1918, Slovenes formed Yugoslavia along with Serbs and Croats, while a minority came under Italy. The state of Slovenia was created in 1945 as part of federal Yugoslavia. Slovenia gained its independence from Yugoslavia in June 1991, and is today a member of the European Union and NATO.'

In [129]:
doc = nlp(text2)
for ent in doc.ents:
  print("{} -> {} ".format(ent.text,ent.label_))

Holy Roman Empire -> GPE 
nearly 1,000 years -> DATE 
between the mid-14th century and 1918 -> DATE 
Slovenia -> GPE 
Habsburg -> GPE 
1918 -> DATE 
Slovenes -> NORP 
Yugoslavia -> GPE 
Serbs -> NORP 
Croats -> NORP 
Italy -> GPE 
Slovenia -> GPE 
1945 -> DATE 
Yugoslavia -> GPE 
Slovenia -> GPE 
Yugoslavia -> GPE 
June 1991 -> DATE 
today -> DATE 
the European Union -> ORG 
NATO -> ORG 


In [130]:
from spacy import displacy 
displacy.render(doc, style='dep',jupyter=True)

In [131]:
# Write a pattern that matches a form of "download" plus proper noun
pattern = [{'POS': 'NUM'},  {'IS_PUNCT': True},{'POS': 'PROPN'}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add('matching_pattern', None, pattern)
matches = matcher(doc)
print('Total matches found:', len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print('Match found:', doc[start:end].text)

Total matches found: 1
Match found: 1918, Slovenes


In [132]:
for token in doc:
    print(token.text,'->',token.dep_,'->',token.pos_)

The -> det -> DET
Holy -> compound -> PROPN
Roman -> compound -> PROPN
Empire -> nsubj -> PROPN
controlled -> ROOT -> VERB
the -> det -> DET
land -> dobj -> NOUN
for -> prep -> ADP
nearly -> advmod -> ADV
1,000 -> nummod -> NUM
years -> pobj -> NOUN
, -> punct -> PUNCT
and -> cc -> CCONJ
between -> conj -> ADP
the -> det -> DET
mid-14th -> nummod -> ADJ
century -> pobj -> NOUN
and -> cc -> CCONJ
1918 -> conj -> NUM
most -> nsubj -> ADJ
of -> prep -> ADP
Slovenia -> pobj -> PROPN
was -> conj -> AUX
under -> prep -> ADP
Habsburg -> compound -> PROPN
rule -> pobj -> NOUN
. -> punct -> PUNCT
In -> prep -> ADP
1918 -> pobj -> NUM
, -> punct -> PUNCT
Slovenes -> nsubj -> PROPN
formed -> ROOT -> VERB
Yugoslavia -> dobj -> PROPN
along -> prep -> ADP
with -> prep -> ADP
Serbs -> pobj -> PROPN
and -> cc -> CCONJ
Croats -> conj -> NOUN
, -> punct -> PUNCT
while -> mark -> SCONJ
a -> det -> DET
minority -> nsubj -> NOUN
came -> advcl -> VERB
under -> prep -> ADP
Italy -> pobj -> PROPN
. -> punct -

In [133]:
#from spacy.lang.en import English # updated

#nlp = English()
#nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
#doc = nlp(text2)
#sentences = [sent.string.strip() for sent in doc.sents]

In [134]:
#sentences

In [152]:
#sent[0].pos_ for sent in doc.sents
#c = [token.pos_ for token in sent for sent in doc.sents]

final_list = []
for sent in doc.sents:
  #print(sent)
  final_list.append([[token.text, token.pos_] for token in sent])

In [153]:
final_list

[[['The', 'DET'],
  ['Holy', 'PROPN'],
  ['Roman', 'PROPN'],
  ['Empire', 'PROPN'],
  ['controlled', 'VERB'],
  ['the', 'DET'],
  ['land', 'NOUN'],
  ['for', 'ADP'],
  ['nearly', 'ADV'],
  ['1,000', 'NUM'],
  ['years', 'NOUN'],
  [',', 'PUNCT'],
  ['and', 'CCONJ'],
  ['between', 'ADP'],
  ['the', 'DET'],
  ['mid-14th', 'ADJ'],
  ['century', 'NOUN'],
  ['and', 'CCONJ'],
  ['1918', 'NUM'],
  ['most', 'ADJ'],
  ['of', 'ADP'],
  ['Slovenia', 'PROPN'],
  ['was', 'AUX'],
  ['under', 'ADP'],
  ['Habsburg', 'PROPN'],
  ['rule', 'NOUN'],
  ['.', 'PUNCT']],
 [['In', 'ADP'],
  ['1918', 'NUM'],
  [',', 'PUNCT'],
  ['Slovenes', 'PROPN'],
  ['formed', 'VERB'],
  ['Yugoslavia', 'PROPN'],
  ['along', 'ADP'],
  ['with', 'ADP'],
  ['Serbs', 'PROPN'],
  ['and', 'CCONJ'],
  ['Croats', 'NOUN'],
  [',', 'PUNCT'],
  ['while', 'SCONJ'],
  ['a', 'DET'],
  ['minority', 'NOUN'],
  ['came', 'VERB'],
  ['under', 'ADP'],
  ['Italy', 'PROPN'],
  ['.', 'PUNCT']],
 [['The', 'DET'],
  ['state', 'NOUN'],
  ['of', 'ADP']

In [202]:
event_list =['PROPN','NUM']
count = 0
for l in final_list:
  #print(l[:])
  l1 = [s[1] for s in l ]
  #print(l1)
  if 'PROPN' in l1 and 'NUM' in l1 :
    count = count+1

print('Number of events: ',count)

['DET', 'PROPN', 'PROPN', 'PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'ADV', 'NUM', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'NUM', 'ADJ', 'ADP', 'PROPN', 'AUX', 'ADP', 'PROPN', 'NOUN', 'PUNCT']
['ADP', 'NUM', 'PUNCT', 'PROPN', 'VERB', 'PROPN', 'ADP', 'ADP', 'PROPN', 'CCONJ', 'NOUN', 'PUNCT', 'SCONJ', 'DET', 'NOUN', 'VERB', 'ADP', 'PROPN', 'PUNCT']
['DET', 'NOUN', 'ADP', 'PROPN', 'AUX', 'VERB', 'ADP', 'NUM', 'SCONJ', 'NOUN', 'ADP', 'ADJ', 'PROPN', 'PUNCT']
['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'NUM', 'PUNCT', 'CCONJ', 'AUX', 'NOUN', 'DET', 'NOUN', 'ADP', 'DET', 'PROPN', 'PROPN', 'CCONJ', 'PROPN', 'PUNCT']
Number of events:  4


In [204]:


fl = []
for sent in doc.sents:
  #print(sent)
  fl.append([[ent.text, ent.label_] for ent in sent.ents])




 # doc = nlp(text2)
#for ent in doc.ents:
#  print("{} -> {} ".format(ent.text,ent.label_))

In [205]:
print(fl)

[[['Holy Roman Empire', 'GPE'], ['nearly 1,000 years', 'DATE'], ['between the mid-14th century and 1918', 'DATE'], ['Slovenia', 'GPE'], ['Habsburg', 'GPE']], [['1918', 'DATE'], ['Slovenes', 'NORP'], ['Yugoslavia', 'GPE'], ['Serbs', 'NORP'], ['Croats', 'NORP'], ['Italy', 'GPE']], [['Slovenia', 'GPE'], ['1945', 'DATE'], ['Yugoslavia', 'GPE']], [['Slovenia', 'GPE'], ['Yugoslavia', 'GPE'], ['June 1991', 'DATE'], ['today', 'DATE'], ['the European Union', 'ORG'], ['NATO', 'ORG']]]


In [206]:
count = 0
for l in fl:
  #print(l[:])
  l1 = [s[1] for s in l ]
  #print(l1)
  if 'GPE' in l1 and 'DATE' in l1 :
    count = count+1

print('Number of events: ',count)

Number of events:  4


event_list =['PROPN','NUM']
count = 0
for l in final_list:
  #print(l[:])
  l1 = [s for s in l ]
  print(l1)
  